In [2]:
from qiskit_optimization.problems import QuadraticProgram
qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.minimize(linear = {'x':2,'y':2}, quadratic = {('x','y'):-3})
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 2 x + 2 y + [ - 6 x*y ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1

Binaries
 x y
End



In [5]:
from qiskit_optimization.algorithms import GroverOptimizer
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
seed = 1234
algorithm_globals.random_seed = seed
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"),
shots = 1024, seed_simulator = seed, seed_transpiler=seed)
grover_optimizer = GroverOptimizer(num_value_qubits = 3, num_iterations=2,
quantum_instance=quantum_instance)

In [6]:
results = grover_optimizer.solve(qp)
print(results)

fval=0.0, x=0.0, y=0.0, status=SUCCESS
